In [6]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os

In [7]:
# style to use for plots
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

In [8]:
df = pd.read_csv('../data/raw_listings.csv')
print(f"Raw data shape: {df.shape}")
df.head()

df.info()
df.describe(include='all')

Raw data shape: (800, 11)
<class 'pandas.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Title          800 non-null    str  
 1   Property Type  800 non-null    str  
 2   Price          800 non-null    str  
 3   Location       800 non-null    str  
 4   Bedrooms       800 non-null    str  
 5   Bathrooms      764 non-null    str  
 6   Size           536 non-null    str  
 7   Amenities      774 non-null    str  
 8   Surroundings   719 non-null    str  
 9   Created At     800 non-null    str  
 10  URL            800 non-null    str  
dtypes: str(11)
memory usage: 375.6 KB


,Title,Property Type,Price,Location,Bedrooms,Bathrooms,Size,Amenities,Surroundings,Created At,URL
count,800,800,800,800,800,764,536,774,719,800,800
unique,434,3,191,77,10,10,184,633,125,85,800
top,5 Bed Townhouse with En Suite in Lavington,House,"KSh 85,000,000",Lavington,5 Bedrooms,5 Bathrooms,450 sqm,"Alarm, Backup Generator, En Suite, Fibre Inter...","Bus Stop, Golf Course, Hospital, Scenic View, ...",17 February 2026,https://www.buyrentkenya.com/listings/3-bedroo...
freq,81,334,37,211,327,229,24,40,182,94,1


In [9]:
# Check for duplicates
print(f"Duplicate URLs: {df['URL'].duplicated().sum()}")
print(f"Duplicate (Title + Price): {df.duplicated(subset=['Title', 'Price']).sum()}")

Duplicate URLs: 0
Duplicate (Title + Price): 136


In [10]:
# ==============================================
#  Remove Duplicates
# We'll drop based on URL (most reliable), then check title+price.


df = df.drop_duplicates(subset=['URL'], keep='first')
print(f"Shape after removing URL duplicates: {df.shape}")

# Optional: also remove title+price duplicates if URL wasn't enough
df = df.drop_duplicates(subset=['Title', 'Price'], keep='first')
print(f"Shape after removing Title+Price duplicates: {df.shape}")


# ### 3. Handle Missing Values
# First, see which columns have missing data.


Shape after removing URL duplicates: (800, 11)
Shape after removing Title+Price duplicates: (664, 11)


In [11]:
#  CHECK AND HANDLE MISSING VALUES
missing = df.isnull().sum()
missing[missing > 0]


# drop rows with missing critical info (like Price or Location). For other columns, we can decide on a case-by-case basis.
df = df.dropna(subset=['Price', 'Location'])
print(f"Shape after dropping rows with missing Price or Location: {df.shape}")

# for bedrooms/bathrooms/size, fill "N/A" with None for now, then convert
df['Bedrooms'] = df['Bedrooms'].replace('N/A', np.nan)
df['Bathrooms'] = df['Bathrooms'].replace('N/A', np.nan)
df['Size'] = df['Size'].replace('N/A', np.nan)

Shape after dropping rows with missing Price or Location: (664, 11)


In [12]:
# =====================================================
#  STANDARDIZE LOCATION NAMES
# standardize location names by extracting main area name and removing common suffixes like "Area", "Estate", etc.

def clean_location(loc):
    if pd.isna(loc):
        return np.nan
    # Remove common suffixes and extra spaces
    loc = str(loc)
    # Extract the main area name (usually the first part before comma)
    main_area = loc.split(',')[0].strip()
    # Remove words like "Area", "Estate", "Town" if they appear (optional)
    main_area = re.sub(r'\s+(Area|Estate|Town)$', '', main_area, flags=re.IGNORECASE)
    return main_area

df['Location_Clean'] = df['Location'].apply(clean_location)
# Check the mapping
df[['Location', 'Location_Clean']].drop_duplicates().head(20)


,Location,Location_Clean
0,"Gikambura, Kikuyu",Gikambura
1,Kiambu Road,Kiambu Road
2,Lavington,Lavington
4,Malindi,Malindi
5,"Runda, Westlands",Runda
7,"Nyari, Westlands",Nyari
8,Karen Hardy,Karen Hardy
9,Karen,Karen
10,Kitengela,Kitengela
13,"Rosslyn, Westlands",Rosslyn


In [13]:
# =====================================================
#  CONVERT PRICE TO NUMERIC
# Remove currency symbols and commas, then convert to float

def clean_price(price_str):
    if pd.isna(price_str):
        return np.nan
    # Remove "KSh", commas, and any non-numeric characters except decimal point
    price_clean = re.sub(r'[^\d.]', '', price_str)
    try:
        return float(price_clean)
    except:
        return np.nan

df['Price_Numeric'] = df['Price'].apply(clean_price)
print(df[['Price', 'Price_Numeric']].head(10))

# Drop rows where price couldn't be parsed
df = df.dropna(subset=['Price_Numeric'])
print(f"Shape after dropping invalid prices: {df.shape}")


             Price  Price_Numeric
0   KSh 22,500,000     22500000.0
1   KSh 78,000,000     78000000.0
2  KSh 160,000,000    160000000.0
3  KSh 130,000,000    130000000.0
4   KSh 21,000,000     21000000.0
5  KSh 135,000,000    135000000.0
6  KSh 130,000,000    130000000.0
7  KSh 130,000,000    130000000.0
8  KSh 120,000,000    120000000.0
9   KSh 80,000,000     80000000.0
Shape after dropping invalid prices: (662, 13)


In [ ]:
#  CONVERT SIZE TO NUMERIC

def extract_size_from_text(text):
    # from messy real estate descriptions, extract realistic built-up/property size... (extraction.ipynb)
    if not text:
        return "N/A"

    text = text.replace(",", "")
    candidates = []

    # ---------------------------------------------------
    # 1. Ranges in sqm (e.g. 350 – 400 sqm, 465 to 476 sqm)
    range_matches = re.findall(
        r'(\d+(\.\d+)?)\s*(?:–|-|to)\s*(\d+(\.\d+)?)\s*(sqm|m²|square meters?)',
        text,
        re.IGNORECASE
    )

    for match in range_matches:
        low = float(match[0])
        high = float(match[2])
        if high >= 30:  # realistic lower bound for Kenya
            candidates.append((high, f"{match[0]}–{match[2]} sqm"))

    # ---------------------------------------------------
    # 2. Single sqm values

    sqm_matches = re.findall(
        r'(\d+(\.\d+)?)\s*(sqm|m²|square meters?)',
        text,
        re.IGNORECASE
    )

    for match in sqm_matches:
        value = float(match[0])
        if value >= 30:  # ~ small studio size
            candidates.append((value, f"{match[0]} sqm"))

    # ---------------------------------------------------
    # 3. Square feet (convert to sqm for comparison)
    
    sqft_matches = re.findall(
        r'(\d+(\.\d+)?)\s*(sq\.?\s*ft\.?|sqft)',
        text,
        re.IGNORECASE
    )

    for match in sqft_matches:
        sqft_value = float(match[0])
        if sqft_value >= 300:  # adjusted for Kenya
            sqm_equivalent = sqft_value * 0.092903
            candidates.append((sqm_equivalent, f"{match[0]} sq ft"))

    # ---------------------------------------------------
    # 4. Acres (convert to sqm for comparison)

    acre_matches = re.findall(
        r'(\d+/\d+|\d+(\.\d+)?)\s*-?\s*(acre)',
        text,
        re.IGNORECASE
    )

    for match in acre_matches:
        raw_value = match[0]

        if "/" in raw_value:
            num, denom = raw_value.split("/")
            acre_value = float(num) / float(denom)
        else:
            acre_value = float(raw_value)

        if acre_value >= 0.05:  # ignore unrealistic tiny plots
            sqm_equivalent = acre_value * 4046.86
            candidates.append((sqm_equivalent, f"{raw_value} acre"))

    # ---------------------------------------------------
    # 5. Return largest realistic size, and avoid small sizes that are likely not the property size
    if candidates:
        # Filter out candidates that are too small (e.g., less than 30 sqm)
        filtered_candidates = [c for c in candidates if c[0] >= 30]
        if filtered_candidates:
            return max(filtered_candidates, key=lambda x: x[0])[1]

    return "N/A"


def extract_size_sqm(text):
"""Return size in square meters as float, or np.nan."""
if pd.isna(text):
    return np.nan
text = str(text).replace(',', '')
# First, handle acres (special case)
acre_match = re.search(r'(\d+/\d+|\d+(\.\d+)?)\s*-?\s*(acre)', text, re.IGNORECASE)
if acre_match:
    raw = acre_match.group(1)
    if '/' in raw:
        num, den = raw.split('/')
        acres = float(num)/float(den)
    else:
        acres = float(raw)
    return acres * 4046.86

# For sqm and sqft, we can use the same regex but convert
# Let's attempt to extract any number followed by sqm/m²
sqm_match = re.search(r'(\d+(\.\d+)?)\s*(sqm|m²|square meters?)', text, re.IGNORECASE)
if sqm_match:
    return float(sqm_match.group(1))

sqft_match = re.search(r'(\d+(\.\d+)?)\s*(sq\.?\s*ft\.?|sqft)', text, re.IGNORECASE)
if sqft_match:
    sqft = float(sqft_match.group(1))
    return sqft * 0.092903

# If we have a range, take the high end (as before)
range_match = re.search(r'(\d+(\.\d+)?)\s*(?:–|-|to)\s*(\d+(\.\d+)?)\s*(sqm|m²|square meters?)', text, re.IGNORECASE)
if range_match:
    high = float(range_match.group(3))
    return high

return np.nan


df['Size_SQM'] = df['Size'].apply(extract_size_sqm)
print(df[['Size', 'Size_SQM']].head(20))

# Drop extreme outliers (e.g., < 10 sqm or > 2000 sqm – adjust as needed)
df = df[(df['Size_SQM'] >= 10) & (df['Size_SQM'] <= 2000)]
print(f"Shape after filtering size outliers: {df.shape}")


IndentationError: expected an indented block after function definition on line 87 (594731631.py, line 88)

In [ ]:
# ==================================================================
#  Create New Features

# %%
#  price_per_sqm
df['Price_per_SQM'] = df['Price_Numeric'] / df['Size_SQM']

# amenity_score (count of amenities)
df['Amenity_Count'] = df['Amenities'].apply(lambda x: len(x) if isinstance(x, list) else 0)

# month and year from Created At
def parse_date(date_str):
    if pd.isna(date_str):
        return pd.NaT
    try:
        # Format like "09 February 2026"
        return datetime.strptime(date_str, '%d %B %Y')
    except:
        return pd.NaT

df['Created_Date'] = df['Created At'].apply(parse_date)
df['Month'] = df['Created_Date'].dt.month
df['Year'] = df['Created_Date'].dt.year

# bedrooms/bathrooms as integers...(incase there is a text like "3 Bedrooms" or "2 Bathrooms", extract the number)
def extract_number(text):
    if pd.isna(text):
        return np.nan
    match = re.search(r'\d+', str(text))
    return int(match.group()) if match else np.nan

df['Bedrooms_Num'] = df['Bedrooms'].apply(extract_number)
df['Bathrooms_Num'] = df['Bathrooms'].apply(extract_number)


In [ ]:
# ===========================================================
# Basic EDA

# %%
# Summary statistics for numeric columns
df[['Price_Numeric', 'Size_SQM', 'Price_per_SQM', 'Bedrooms_Num', 'Bathrooms_Num', 'Amenity_Count']].describe()

# %%
# Distribution of prices
plt.figure()
sns.histplot(df['Price_Numeric'], bins=50, kde=True)
plt.title('Distribution of House Prices (KES)')
plt.xlabel('Price (KES)')
plt.tight_layout()
plt.savefig('../reports/price_distribution.png')
plt.show()

# %%
# Top locations by median price
top_locs = df.groupby('Location_Clean')['Price_Numeric'].median().sort_values(ascending=False).head(15)
plt.figure()
top_locs.plot(kind='bar')
plt.title('Top 15 Locations by Median Price')
plt.ylabel('Median Price (KES)')
plt.tight_layout()
plt.savefig('../reports/top_locations.png')
plt.show()

# %%
# Correlation heatmap
numeric_cols = ['Price_Numeric', 'Size_SQM', 'Bedrooms_Num', 'Bathrooms_Num', 'Amenity_Count', 'Price_per_SQM']
plt.figure(figsize=(8,6))
sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.tight_layout()
plt.savefig('../reports/correlation.png')
plt.show()

In [ ]:
# ======================================================
# SAVE CLEANED DATASET

# %%
# Select relevant columns for modeling
clean_df = df[['Title', 'Property Type', 'Location_Clean', 'Bedrooms_Num', 'Bathrooms_Num',
               'Size_SQM', 'Amenities', 'Surroundings', 'Price_Numeric', 'Price_per_SQM',
               'Amenity_Count', 'Month', 'Year', 'URL']].copy()
clean_df.rename(columns={'Location_Clean': 'Location'}, inplace=True)

# Save
os.makedirs('../data', exist_ok=True)
clean_df.to_csv('../data/clean_listings.csv', index=False)
print("Clean data saved to ../data/clean_listings.csv")
